In [2]:
import pandas as pd
import geopy as geo
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler # Handliing Feature Scaling
from sklearn.impute import SimpleImputer #Handling missing values
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding
##Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [45]:
data = pd.read_csv("https://raw.githubusercontent.com/AKISHPOTHURI/DataScience/main/Dataset/online_order.csv")
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [46]:
data.isnull().sum()

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [33]:
data=data.drop(labels=['ID','Delivery_person_ID'], axis=1)
df = data.loc[data.isnull().sum(1)>=3]
data1 = pd.concat([data, df, df]).drop_duplicates(keep=False)
df = data1[data1['City'].notna()]

In [ ]:
for i in df.index:
    if df['Time_Orderd'][i] >= '05:00' and df['Time_Orderd'][i] < '10:00':
        df.loc[i,'day_quaters'] = 'morning'
    elif df['Time_Orderd'][i] >= '10:00' and df['Time_Orderd'][i] < '14:00':
        df.loc[i,'day_quaters'] = 'late morning'
    elif df['Time_Orderd'][i] >= '14:00' and df['Time_Orderd'][i] < '19:00':
        df.loc[i,'day_quaters'] = 'afternoon'
    elif df['Time_Orderd'][i] >= '19:00':
        df.loc[i,'day_quaters'] = 'night'
df=df.drop(labels=['Time_Orderd','Time_Order_picked','Order_Date'], axis=1)

In [63]:
# df = df[df['Delivery_person_Age'].notna()]
# df = df[df['Delivery_person_Ratings'].notna()]
# df = df[df['multiple_deliveries'].notna()]
# df = df[df['Festival'].notna()]
df['Festival'].fillna("No", inplace = True) # 98% of the column has No as a value 
df['day_quaters'].fillna("night", inplace = True) # most of the people ordered at night
df['Delivery_person_Ratings'].fillna(round(np.mean(df['Delivery_person_Ratings']),1), inplace = True)
df['Delivery_person_Age'].fillna(round(np.mean(df['Delivery_person_Age'])), inplace = True)
df['multiple_deliveries'].fillna(round(np.mean(df['multiple_deliveries'])), inplace=True)

In [65]:
df.isnull().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
day_quaters                    0
dtype: int64

In [76]:
df = df.dropna()

In [77]:

# Set the earth's radius (in kilometers)
R = 6371
index = []
# Convert degrees to radians
def deg_to_rad(degrees):
    return degrees * (np.pi/180)

# Function to calculate the distance between two points using the haversine formula
def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c
  
# Calculate the distance between each pair of points
# data1['distance'] = np.nan

for i in range(len(df)):
    if i in data1.index:
        df.loc[i, 'distance'] = distcalculate(df.loc[i, 'Restaurant_latitude'], 
                                        df.loc[i, 'Restaurant_longitude'], 
                                        df.loc[i, 'Delivery_location_latitude'], 
                                        df.loc[i, 'Delivery_location_longitude'])
    else:
        index.append(i)

In [78]:
#Independent and dependent features
X = df.drop(labels=['Time_taken (min)'], axis=1)
Y = df[['Time_taken (min)']]

In [79]:
X

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,day_quaters,distance
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,night,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,afternoon,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,afternoon,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,morning,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,night,19.396618
5,29.0,4.5,19.176269,72.836721,19.266269,72.926721,Sandstorms,Jam,2,Buffet,electric_scooter,1.0,No,Metropolitian,night,13.763977
6,35.0,4.0,12.311072,76.654878,12.351072,76.694878,Windy,High,1,Meal,scooter,1.0,No,Metropolitian,afternoon,6.218001
7,33.0,4.2,18.592718,73.773572,18.702718,73.883572,Sandstorms,Jam,2,Snack,motorcycle,1.0,No,Metropolitian,night,16.849940
8,34.0,4.9,17.426228,78.407495,17.496228,78.477495,Cloudy,Jam,0,Snack,motorcycle,1.0,No,Metropolitian,night,10.757109
9,21.0,4.7,22.552672,88.352885,22.582672,88.382885,Windy,Jam,0,Meal,motorcycle,1.0,No,Urban,night,4.540574


In [80]:
#Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [81]:
df.dtypes

Delivery_person_Age            float64
Delivery_person_Ratings        float64
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Weather_conditions              object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries            float64
Festival                        object
City                            object
Time_taken (min)                 int64
day_quaters                     object
distance                       float64
dtype: object

In [82]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object')

In [83]:
city_map = {'Metropolitian','Urban','Semi-Urban'}
Festival_map = {'Yes','No'}
Road_traffic_density_map = {'Jam','High','Medium','Low'}
Type_of_order_map = {'Snack','Meal','Drinks','Buffet'}
Type_of_vehicle_map = {'motorcycle','scooter','electric_scooter'}
Weather_conditions_map = {'Fog','Stormy','Sandstorms','Windy','Cloudy','Sunny'}
day_quaters_map = {'night','afternoon','morning','late morning'}

In [84]:
#Nuerical Pipeline
num_pipeline = Pipeline(
    steps=[
    # ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ]
)

#Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OrdinalEncoder', OrdinalEncoder(categories=[Weather_conditions_map,Road_traffic_density_map,Type_of_order_map,Type_of_vehicle_map,Festival_map,
                                                  city_map,day_quaters_map])),
    ('scaler', StandardScaler())
    ]
)

In [85]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object')

In [86]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [90]:
#Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.30,random_state=42)

In [91]:
X_train= pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [27]:
categorical_cols

Index(['Delivery_person_ID', 'Order_Date', 'Time_Orderd', 'Time_Order_picked',
       'Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')

In [28]:
numerical_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries'],
      dtype='object')

In [29]:
X.isnull().sum()

Delivery_person_ID               0
Delivery_person_Age            197
Delivery_person_Ratings        250
Restaurant_latitude              0
Restaurant_longitude             0
Delivery_location_latitude       0
Delivery_location_longitude      0
Order_Date                       0
Time_Orderd                      0
Time_Order_picked                0
Weather_conditions               0
Road_traffic_density             0
Vehicle_condition                0
Type_of_order                    0
Type_of_vehicle                  0
multiple_deliveries            901
Festival                       200
City                             0
dtype: int64